# GremlinServerConnection


This example notebook shows how to persist graph in a gremlin server.

Inspired by examples of Michael Pollmeier [https://github.com/mpollmeier/gremlin-scala-examples/blob/master/gremlin-server/src/test/scala/SimpleSpec.scala]<br>

<div><p>The target gremlin server is configured with Neo4j as graph storage, however we only use gremlin websocket connection in that example, nothing here requires having a Neo4j graph listening the bolt protocol. Compatibility with Cypher and integration of Standard Neo4j driver will be shown in
 another Notebook.</p><p>This example imports serializers that fits Gremlin Server configuration.</p><p>Beware of the misleading name: EmptyGraph, graph data is persisted within Gremlin Server, the function <code>withRemote()</code> ensures the graph is loaded into the so called in-memory-<code>EmptyGraph</code>.</p><p>Declaring the <code>graph</code> variable as <b>implicit </b>to be automatically called by <code>gremlin.scala._</code> throughout the notebook (#justScalaThings).<br></p></div>

In [2]:
import gremlin.scala._

import org.apache.tinkerpop.gremlin.driver.Cluster
import org.apache.tinkerpop.gremlin.driver.remote.DriverRemoteConnection
import org.apache.tinkerpop.gremlin.driver.ser.GryoMessageSerializerV3d0
import org.apache.tinkerpop.gremlin.structure.io.gryo.GryoMapper 
import org.apache.tinkerpop.gremlin.structure.util.empty.EmptyGraph
import org.apache.tinkerpop.gremlin.tinkergraph.structure.TinkerIoRegistryV3d0

val serializer = new GryoMessageSerializerV3d0(
    GryoMapper.build.addRegistry(TinkerIoRegistryV3d0.instance)
)

val cluster = Cluster
                .build
                .addContactPoint("gremlinserver-neo4j")
                .port(8182)
                .serializer(serializer)
                .create

implicit val graph = EmptyGraph.instance.asScala.configure(_.withRemote(DriverRemoteConnection.using(cluster)))

<div>This is not pure programming here: all operations are made against the graph stored in the gremlin server.<br></div>

In [4]:
val Name = Key[String]("name")
val Planet = "planet"

val saturn: Vertex = graph + (Planet, Name -> "saturn")
val sun   : Vertex = graph + (Planet, Name -> "sun")

saturn --- "orbits" --> sun

e[0][0-orbits->1]

In [5]:
val nodes = graph.V.value(Name).toList
val edges = graph.E.toList

<u><b>@TODO Add a working vegaSpec example displaying the gremlin graph !</b></u><br>

In [7]:
cluster.close()